Importing Libraries

After the Google API call we are reformating the output of the API call to Description,Input and Expected Output

In [3]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import markdown
import re
import textwrap
from IPython.display import Markdown

C:\Users\bOct0\AppData\Local\Temp\ipykernel_27304\1258265804.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Markdown as Mentioned in Google API DOCS 

In [13]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

Markdown as by html parser

In [4]:
def toString(input_string):
    # print(input_string)
    html = markdown.markdown(input_string)
    text = BeautifulSoup(html, features='html.parser')
    text = text.get_text().strip().replace('\n',' ')
    text = text.strip().replace('*','')
    # text = re.sub(r'(?<!^)(?=[A-Z])', ' ', text)
    # res = re.sub(' +', ' ', text)
    return text

Converting each test cases to Required Format

In [15]:
def toDataFrame(res,userstories):
    
    # Split the string into individual test cases
    test_cases = res.split("Test Case")[1:]

    # Initialize lists to store data
    descriptions = []
    inputs = []
    expected_outputs = []

    # Extract information for each test case
    for test_case in test_cases:
        parts = test_case.split("Description")
        # print(parts)
        if "Description" in test_case:
            description = parts[1].split("Input:")[0].strip()
            input_part = parts[1].split("Input:")[1]
            input_data = input_part.split("Expected Output:")[0].strip()
            expected_output = input_part.split("Expected Output:")[1].strip()
            
            descriptions.append(description)
            inputs.append(input_data)
            expected_outputs.append(expected_output)

    # Create a dataframe
    data = {
        "User Story":userstories,
        "Description": descriptions,
        "Input": inputs,
        "Expected Output": expected_outputs
    }
    return data

In [5]:
data = pd.DataFrame(columns=["User Story","Description","Input","Expected Output"])

In [20]:
excel_file = 'train.xlsx'
df = pd.read_excel(excel_file)

In [23]:
# Initialize lists to store data
descriptions = []
inputs = []
expected_outputs = []
story =[]
for index, row in df.iterrows():
    user_story = df['userstories'][index]
    input_string = df['response'][index]
    input_string = toString(input_string)
        
    # Split the string into individual test cases
    test_cases = input_string.split("Test Case")[1:]    
    # Extract information for each test case
    for test_case in test_cases:
        parts = test_case.split("Description")
        # print(parts)
        if "Description" in test_case:
            description = parts[1].split("Input")[0].strip()
            # print(description)
            if "Input" in test_case:
                input_part = parts[1].split("Input")[1].strip()
                input_data = input_part.split("Expected Output")[0].strip()
                print(input_part)
                if "Expected Output" in test_case:
                    expected_output = input_part.split("Expected Output")[1].strip()
                    # print(index+1)
                story.append(user_story)
                descriptions.append(description)
                inputs.append(input_data)
                expected_outputs.append(expected_output)
            
    # Create a dataframe
    data = {
        "User Story":story,
        "Description": descriptions,
        "Input": inputs,
        "Expected Output": expected_outputs
    }

:      Record ID: 12345  Expected Output:      Record 12345 is deleted from the database.
:      Record IDs: 12345, 67890, 98765  Expected Output:      Records 12345, 67890, and 98765 are deleted from the database.
:      Record ID: 12345      Child Record IDs: 54321, 65432  Expected Output:      Record 12345 and its child records (54321 and 65432) are deleted from the database.
:      Record ID: 12345      Referencing Record ID: 67890  Expected Output:      Record 12345 cannot be deleted because it is referenced by record 67890.      An error message is generated.
:      Record ID: 99999  Expected Output:      Record 99999 does not exist in the database.      An error message is generated.
: Resources page with updated header Expected Output: Header displays in the new Broker design style, with appropriate fonts, colors, and spacing
: Resources page with updated content blocks Expected Output: Content blocks are arranged in a logical and visually appealing manner, following the new Br

In [24]:
d = pd.DataFrame(data)
d

,User Story,Description,Input,Expected Output
0,I want to have the 12-19-2017 deletions proce...,: Process the deletion of a single record.,: Record ID: 12345,: Record 12345 is deleted from the database.
1,I want to have the 12-19-2017 deletions proce...,: Process the deletion of multiple records.,": Record IDs: 12345, 67890, 98765",": Records 12345, 67890, and 98765 are del..."
2,I want to have the 12-19-2017 deletions proce...,: Process the deletion of a record with child ...,: Record ID: 12345 Child Record IDs:...,: Record 12345 and its child records (543...
3,I want to have the 12-19-2017 deletions proce...,: Process the deletion of a record that is ref...,: Record ID: 12345 Referencing Recor...,: Record 12345 cannot be deleted because ...
4,I want to have the 12-19-2017 deletions proce...,: Process the deletion of a record that does n...,: Record ID: 99999,: Record 99999 does not exist in the data...
...,...,...,...,...
8164,"I want to see a summary of articles, so that ...",Verify that the article summary is generated c...,": An article with the title ""How to Write Good",": List of general volunteer projects, e.g. ""Gr..."
8165,"I want to see a summary of articles, so that ...",Verify that the article summary is generated c...,": A list of articles with the titles ""How to W...",": List of general volunteer projects, e.g. ""Gr..."
8166,"I want to see a summary of articles, so that ...",Verify that the article summary is generated c...,": An article with the title ""A Comprehensive G...",": List of general volunteer projects, e.g. ""Gr..."
8167,"I want to see a summary of articles, so that ...",Verify that the article summary is generated c...,": An article with the title ""Advanced Techniqu...",: A summary that includes the key points of th...


In [26]:
d.to_excel("final.xlsx")

END